In [1]:
import json
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
import trt_pose.coco
import math
import os
import numpy as np
import traitlets
import pickle 


In [2]:
%cd ..
%pwd

/home/project


'/home/project'

In [3]:
with open('preprocess/hand_pose.json', 'r') as f:
    hand_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(hand_pose)
import trt_pose.models

num_parts = len(hand_pose['keypoints'])
num_links = len(hand_pose['skeleton'])

model = trt_pose.models.resnet18_baseline_att(num_parts, 2 * num_links).cuda().eval()
import torch


WIDTH = 224
HEIGHT = 224
data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()

if not os.path.exists('model/hand_pose_resnet18_att_244_244_trt.pth'):
    MODEL_WEIGHTS = 'model/hand_pose_resnet18_att_244_244.pth'
    model.load_state_dict(torch.load(MODEL_WEIGHTS))
    import torch2trt
    model_trt = torch2trt.torch2trt(model, [data], fp16_mode=True, max_workspace_size=1<<25)
    OPTIMIZED_MODEL = 'model/hand_pose_resnet18_att_244_244_trt.pth'
    torch.save(model_trt.state_dict(), OPTIMIZED_MODEL)


OPTIMIZED_MODEL = 'model/hand_pose_resnet18_att_244_244_trt.pth'
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

<All keys matched successfully>

In [4]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology,cmap_threshold=0.12, link_threshold=0.15)
draw_objects = DrawObjects(topology)

In [5]:

import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [6]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto', kernel='rbf'))

In [7]:
print(topology)
print(num_parts)

tensor([[ 0,  1,  0,  4],
        [ 2,  3,  0,  8],
        [ 4,  5,  0, 12],
        [ 6,  7,  0, 16],
        [ 8,  9,  0, 20],
        [10, 11,  1,  2],
        [12, 13,  2,  3],
        [14, 15,  3,  4],
        [16, 17,  5,  6],
        [18, 19,  6,  7],
        [20, 21,  7,  8],
        [22, 23,  9, 10],
        [24, 25, 10, 11],
        [26, 27, 11, 12],
        [28, 29, 13, 14],
        [30, 31, 14, 15],
        [32, 33, 15, 16],
        [34, 35, 17, 18],
        [36, 37, 18, 19],
        [38, 39, 19, 20]], dtype=torch.int32)
21


In [8]:
from preprocessdata import preprocessdata
preprocessdata = preprocessdata(topology, num_parts)

In [9]:
from dataloader import dataloader
path = "data/"
label_file = "hand_dataset_train_takechi.json"
test_label = "hand_dataset_test_takechi.json"
hand = dataloader(path, label_file, test_label)

In [30]:
def data_preprocess(images):
    dist_bn_joints_all_data = []
    for idx, im in enumerate(images):
        print(idx)
        #im = im[:, ::-1, :]
        data_im = preprocess(im)
        print('data_im:', data_im)
        cmap, paf = model_trt(data_im)
        cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
        print('cmap_size:', cmap.size())
        print('cmap:', cmap)
        print('paf_size:', paf.size())
        print('paf:', paf)
        counts, objects, peaks = parse_objects(cmap, paf)
        print('counts:', counts)
        print('objects:', objects)
        print('peaks:', peaks)
        img = PIL.Image.fromarray(im)
        #display(img)
        draw_objects(im, counts, objects, peaks)
        joints = preprocessdata.joints_inference(im, counts, objects, peaks)
        print('joints:', joints)
        dist_bn_joints = preprocessdata.find_distance(joints)
        print('dist_bn_joints', dist_bn_joints)
        dist_bn_joints_all_data.append(dist_bn_joints)
    return dist_bn_joints_all_data

In [31]:
def smaller_dataset(dataset, no_samples_per_class, no_of_classes):
    total_samples_per_class =100
    start = 0
    end = no_samples_per_class
    new_dataset = []
    labels = []
    for i in range(no_of_classes):
        new_data = dataset[start:end]
        start = start+total_samples_per_class
        end = start+no_samples_per_class
        new_dataset.extend(new_data)
        labels.extend([i+1]*no_samples_per_class)
    return new_dataset, labels

In [32]:
train_images, labels_train = hand.smaller_dataset(hand.train_images,100,2)

In [33]:
joints_train = data_preprocess(train_images)
joints_test = data_preprocess(hand.test_images)

0
data_im: tensor([[[[ 0.0912,  0.0912,  0.0912,  ...,  0.0741,  0.0741,  0.0569],
          [ 0.0912,  0.0912,  0.0912,  ...,  0.0569,  0.0569,  0.0398],
          [ 0.0912,  0.0912,  0.0912,  ...,  0.0569,  0.0569,  0.0398],
          ...,
          [ 0.7077,  0.7419,  0.7591,  ..., -1.2959, -1.3302, -1.3644],
          [ 0.7077,  0.7419,  0.7591,  ..., -1.2959, -1.3302, -1.2959],
          [ 0.7591,  0.7762,  0.7762,  ..., -1.1760, -1.3302, -1.3473]],

         [[ 0.3627,  0.3627,  0.3627,  ...,  0.2402,  0.2402,  0.2227],
          [ 0.3627,  0.3627,  0.3627,  ...,  0.2227,  0.2227,  0.2052],
          [ 0.3627,  0.3627,  0.3627,  ...,  0.2227,  0.2227,  0.2227],
          ...,
          [ 0.8880,  0.9230,  0.9405,  ..., -1.3354, -1.4405, -1.4755],
          [ 0.8880,  0.9230,  0.9405,  ..., -1.3354, -1.4405, -1.4230],
          [ 0.9405,  0.9580,  0.9580,  ..., -1.2304, -1.4405, -1.4755]],

         [[ 0.6182,  0.6182,  0.6182,  ...,  0.4439,  0.4439,  0.4265],
          [ 0.6182,

KeyboardInterrupt: 

In [ ]:
print(np.array(joints_train).shape)
print(joints_train)
print(np.array(joints_test).shape)
print(joints_test)

In [ ]:
svm_train = True
if svm_train:
    clf, predicted = preprocessdata.trainsvm(clf, joints_train, joints_test, hand.labels_train, hand.labels_test)
    filename = 'svmmodel_new.sav'
    pickle.dump(clf, open(filename, 'wb'))
else:
    filename = 'svmmodel.sav'
    clf = pickle.load(open(filename, 'rb'))

In [ ]:
preprocessdata.svm_accuracy(clf.predict(joints_test), hand.labels_test)


In [ ]:
clf.predict([joints_test[40],[0]*num_parts*num_parts])

In [ ]:
clf.predict(joints_test)

In [ ]:
from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30, capture_device=0)
#camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=30)

camera.running = True

In [ ]:
import ipywidgets
from IPython.display import display


image_w = ipywidgets.Image(format='jpeg', width=256, height=256)
display(image_w)

In [ ]:
with open('preprocess/gesture.json', 'r') as f:
    gesture = json.load(f)
gesture_type = gesture["paint"]

In [ ]:
def execute(change):
    image = change['new']
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)#, cmap_threshold=0.15, link_threshold=0.15)
    draw_objects(image, counts, objects, peaks)
    joints = preprocessdata.joints_inference(image, counts, objects, peaks)
    dist_bn_joints = preprocessdata.find_distance(joints)
    gesture = clf.predict([dist_bn_joints,[0]*num_parts*num_parts])
    gesture_joints = gesture[0]
    preprocessdata.prev_queue.append(gesture_joints)
    preprocessdata.prev_queue.pop(0)
    preprocessdata.print_label(image, preprocessdata.prev_queue, gesture_type)
    image_w.value = bgr8_to_jpeg(image)


In [ ]:
execute({'new': camera.value})

In [ ]:
camera.observe(execute, names='value')

In [ ]:
camera.unobserve_all()

In [ ]:
#camera.running = False